# Aufgabe 1 - k-NN Klassifikation

## Teilaufgabe a)

## Teilaufgabe b)

## Teilaufgabe c)

In [20]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [29]:
def dist(a,b):
    return np.linalg.norm(a-b)

# Für den Fall dass man etwas auslagern möchte
def makeSortedDistanceList(samplePoint, trainingData, labels):
    distanceList = []
    counter = 0
    for j in trainingData:
        distance = dist(samplePoint, j)
        distanceList.append((distance, labels[counter]))
        counter += 1
    distanceList = np.asarray(distanceList)
    sortedDistanceList = distanceList[distanceList[:,0].argsort()]
    return sortedDistanceList

class KNN:
    '''KNN Classifier.

    Attributes
    ----------
    k : int
        Number of neighbors to consider.
    '''
    def __init__(self, k):
        '''Initialization.
        Parameters are stored as member variables/attributes.
        
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        '''
        self.k = k

    def fit(self, X, y):
        '''Fit routine.
        Training data is stored within object.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Training data.
        y : numpy.array shape=(n_samples)
            Training labels.
        '''
        self.training_data = X
        self.training_labels = y

    def predict(self, X):
        '''Prediction routine.
        Predict class association of each sample of X.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Data to classify.
        
        Returns
        -------
        prediction : numpy.array, shape=(n_samples)
            Predictions, containing the predicted label of each sample.
        '''
        
        # First we need to find a few parameters
        n_attributes, n_samples = X.shape # for whatever reason no parentheses after shape...
        n_training_data = np.size(self.training_data, axis = 1) # this means finding the amount of columns
        
        prediction = []
        
        for i in range(X.shape[0]): # for i in sampleData
            distanceList = []
            kNearestNeighbors = []
            back = 0
            sig = 0
            counter = 0
            a = X[i]
            if len(a.shape) != 2:
                a = a.reshape(1, a.shape[0])
            #print('a hat', a.shape[1], 'Zeilen')
            #print('Die Trainingsdaten haben', self.training_data.shape[1], 'Zeilen')
            #print(self.training_data)
            distanceTest = cdist(a, self.training_data)[0]
            # old code:
            #for j in self.training_data:
            #    distance = dist(X[i], j)
            #    distanceList.append((distance, self.training_labels[counter]))
            #    counter += 1
            #distanceList = np.asarray(distanceList)
            distanceList = np.stack((distanceTest, self.training_labels), axis=-1) # Klebe die beiden Arrays als Spalten zusammen!
            sortedDistanceList = distanceList[distanceList[:,0].argsort()]
            kNearestNeighbors = sortedDistanceList[:self.k]
            for l in range(0, self.k):
                if kNearestNeighbors[l,1] == 0:
                    back += 1
                else:
                    sig +=1
            if back > sig:
                prediction.append(0)
            elif back < sig:
                prediction.append(1)
            else:
                prediction.append(float('NaN'))
        return prediction

## Teilaufgabe d)

In [69]:
lenTraining = 5000
lenSignal = 10000
lenBackground = 20000

hdf = pd.read_hdf('NeutrinoMC.hdf5', key = 'Signal')
hits = hdf.NumberOfHits
x = hdf.x
y = hdf.y

hits = np.asarray(hits)
hits = hits[~np.isnan(hits)] # ~ means not 
x = np.asarray(x)
x = x[~np.isnan(x)]
y = np.asarray(y)
y = y[~np.isnan(y)]

matrix = np.matrix([hits, x, y])
print(matrix[:,0:3]) # Look at the first three columns to see if it worked properly

[[ 26.          38.          87.        ]
 [  6.69494405   7.74364571   7.41714979]
 [  2.20707621   3.57920725   2.96454426]]


In [70]:
trainingDataSignal = matrix[:,0:lenTraining]
testDataSignal = matrix[:,lenTraining:lenTraining+lenSignal]
testDataSignal = np.transpose(testDataSignal)

In [71]:
hdf = pd.read_hdf('NeutrinoMC.hdf5', key = 'Background')
hits = hdf.NumberOfHits
x = hdf.x
y = hdf.y

hits = np.asarray(hits)
hits = hits[~np.isnan(hits)]
x = np.asarray(x)
x = x[~np.isnan(x)]
y = np.asarray(y)
y = y[~np.isnan(y)]

matrix = np.matrix([hits, x, y])
trainingDataBackground = matrix[:,0:lenTraining]
testDataBackground = matrix[:,lenTraining:lenBackground+lenTraining]
testDataBackground = np.transpose(testDataBackground)


trainingData = np.concatenate((trainingDataBackground, trainingDataSignal), axis=1)
trainingData = np.transpose(trainingData)

y1 = np.zeros(lenTraining)
y2 = np.ones(lenTraining)
labels = np.concatenate((y1,y2)) # Lets say 0 is background and 1 is signal

In [72]:
knn = KNN(10)
#knn.fit(trainingData, labels)
#predictionS = knn.predict(testDataSignal)
#predictionB = knn.predict(testDataBackground)

In [73]:
#predictionS = np.asarray(predictionS)
#tp = np.count_nonzero(predictionS == 1)
#fp = np.count_nonzero(predictionS == 0)
#tn = np.count_nonzero(predictionB == 1)
#fn = np.count_nonzero(predictionB == 0)
#Reinheit = tp/(tp+fp)
#Effizienz = tp/(tp+fn)
#Signifikanz = tp/(np.sqrt(tp+fp))
#print('Reinheit =', Reinheit)
#print('Effizienz =', Effizienz)
#print('Signifikanz =', Signifikanz)

In [78]:
knn.fit(trainingData, labels)
%time knn.predict(testDataSignal)

Wall time: 11.5 s


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [75]:
training = np.array([[1,1,2,2,2,5,4,5,5,5], [1,2,1,2,3,2,4,4,5,6]])
training = np.transpose(training)
y = np.array([0,0,0,0,0,1,1,1,1,1])
data = np.array([[2,3,3,3,5], [4,2,1,4,1]])
data = np.transpose(data)
knn = KNN(1)
knn.fit(training,y)
print(knn.predict(data))

[0, 0, 0, 1, 1]
